<a href="https://colab.research.google.com/github/AndrewCrossman/Graduate_Research/blob/main/BNL_2023/EXAFS_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXAFS_Analysis.ipynb

This code will help analyze EXAFS data stored on a shared google drive.

**Capabilities:**


# Imports

In [18]:
from google.colab import drive
import csv
import glob
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
from google.colab import files
import matplotlib.pylab as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


# Mount Google Drive Access

In [3]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
!ls '/content/drive/MyDrive/2023-07-30'

 athena_projects   Cu-2423M.002       experiment.log   Ni-2623C.004
'chi(k)'	   Cu-2423M.003      'mu(E)'	       Ni-2623D.001
'chi(q)'	   Cu-2423M.004       Ni-2326K.001     Ni-2623D.002
'chi(R)'	   Cu-2423N.001       Ni-2326K.002     Ni-2623D.003
 Cu-1223.001	   Cu-2423N.002       Ni-2326K.003     Ni-2623D.004
 Cu-1223.002	   Cu-2423N.003       Ni-2326K.004     Ni-2623E.001
 Cu-1223.003	   Cu-2423N.004       Ni-2423.001      Ni-2623E.002
 Cu-1223.004	   Cu-2423O.001       Ni-2423.002      Ni-2623E.003
 Cu-1223A.001	   Cu-2423O.002       Ni-2423.003      Ni-2623E.004
 Cu-1223A.002	   Cu-2423O.003       Ni-2423.004      Ni-2623F.001
 Cu-1223A.003	   Cu-2423O.004       Ni-2423A.001     Ni-2623F.002
 Cu-1223A.004	   Cu-2623.001	      Ni-2423A.002     Ni-2623F.003
 Cu-1223B.001	   Cu-2623.002	      Ni-2423A.003     Ni-2623F.004
 Cu-1223B.002	   Cu-2623.003	      Ni-2423A.004     Ni-2623G.001
 Cu-1223B.003	   Cu-2623.004	      Ni-2423B.001     Ni-2623G.002
 Cu-1223B.004	   Cu-2623A.001     

# xmu Analysis

In [31]:
directory_path = '/content/drive/MyDrive/2023-07-30/mu(E)'
xmu_files = glob.glob(directory_path + '/*')
sort(xmu_files)
for xmu_file in sort(xmu_files):
  print(xmu_file)
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
  xmu_data = xmu_data.dropna(axis=1, how='all')
xmu_data

/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223A.001.xmu


<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read

/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223A.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223A.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223A.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223B.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223B.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223B.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223B.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223C.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223C.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223C.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223C.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223D.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223D.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223D.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223D.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223E.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-1223E.002.xmu
/content/drive/MyDrive/2023-07-

<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read

/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423A.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423A.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423A.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423A.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423B.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423B.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423B.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423B.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423C.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423C.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423C.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423C.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423D.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423D.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423D.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Cu-2423D.004.xmu
/content/drive/MyDrive/2023-07-3

...

/content/drive/MyDrive/2023-07-30/mu(E)/Ni-2623O.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-2623O.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-2623O.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-2623O.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3223.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3223.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3223.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3223.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3323.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3323.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3323.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3323.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523A.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-

<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read

/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523D.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523D.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523D.004.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523E.001.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523E.002.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523E.003.xmu
/content/drive/MyDrive/2023-07-30/mu(E)/Ni-3523E.004.xmu


<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read_csv(xmu_file, on_bad_lines='skip', skiprows=39, sep="  ")
<ipython-input-31-c9508c10cec3>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  xmu_data = pd.read

,#,Unnamed: 3,Unnamed: 6,bkg,Unnamed: 10,Unnamed: 12
0,8132.834,0.591146,0.591146,-0.000414,1.123540e-06,133.02877
1,8142.817,0.587018,0.587018,-0.000402,1.203099e-06,132.96151
2,8152.829,0.583101,0.583101,-0.000390,7.640411e-07,132.86776
3,8162.926,0.579185,0.579185,-0.000387,-3.743072e-07,132.77718
4,8173.006,0.575293,0.575293,-0.000397,-8.546466e-07,132.68694
...,...,...,...,...,...,...
348,8953.272,0.608641,0.615048,-0.000320,2.272774e-06,117.53356
349,8958.134,0.607166,0.614016,-0.000320,-1.746238e-06,117.66584
350,8963.028,0.605521,0.613008,-0.000337,-1.282272e-06,117.52609
351,8967.935,0.603864,0.612031,-0.000332,9.977408e-07,117.40933
